In [1]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset

创建自定义数据集类

In [2]:
from torchvision import transforms

Transform = transforms.Compose([
    transforms.Resize((512, 512)),
    # other transforms, if needed
])

class BreastCancerSegmentationDataset(Dataset):
    """
    乳腺癌分割数据集
    """
    def __init__(self, img_dir, mask_dir, transform=None, one_hot_encode=True, target_size=(256, 256)):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.one_hot_encode = one_hot_encode
        self.target_size = target_size
        self.img_filenames = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, index):
        img_name = self.img_filenames[index]
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name[:-4] + '_mask'+img_name[-4:])
        # Skip .ipynb_checkpoints files
        if img_path.endswith(".ipynb_checkpoints") or mask_path.endswith(".ipynb_checkpoints"):
            return self.__getitem__((index + 1) % len(self))
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            raise FileNotFoundError(f"Image not found at {img_path}")
        if mask is None:
            raise FileNotFoundError(f"Mask not found at {mask_path}")

        # Resize image and mask
        image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_LINEAR)
        mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)

        if self.one_hot_encode:
            mask = one_hot_encode(mask, num_classes=3)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        mask = np.asarray(mask)   # 转换为NumPy数组
        mask = mask.astype(np.float32)   # 变换dtype
        mask = torch.from_numpy(mask) # 转换为Tensor
        return image, mask

class BreastCancerClassificationDataset(Dataset):
    """
    乳腺癌分类数据集
    """
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_filenames = os.listdir(img_dir)

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, index):
        img_name = self.img_filenames[index]
        img_path = os.path.join(self.img_dir, img_name)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)

        if image is None:
            raise FileNotFoundError(f"Image not found at {img_path}")

        if self.transform:
            image = self.transform(image=image)["image"]


        # 从图像名称解析分类标签
        if 'benign' in img_name:
            label = 0
        elif 'malignant' in img_name:
            label = 1
        elif 'normal' in img_name:
            label = 2

        return image, label

def one_hot_encode(mask, num_classes):
    one_hot = np.zeros((mask.shape[0], mask.shape[1], num_classes), dtype=np.uint8)
    for c in range(num_classes):
        one_hot[..., c] = (mask == c)
    return one_hot


使用自定义数据集类加载数据并进行数据增强
划分数据集为训练集和验证集和测试集

In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=30, p=0.3),
    A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.2),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

dataset = {
    'train': BreastCancerSegmentationDataset("image", "mask", transform=train_transform),
    'val': BreastCancerSegmentationDataset("image", "mask", transform=val_transform),
    'test': BreastCancerSegmentationDataset("image", "mask", transform=test_transform)
}

# 打印一个样本，检查数据格式和变换是否符合预期
dataset['train'][10]


(tensor([[[-2.0665,  0.3823, -1.3302,  ...,  1.6667,  1.7523,  1.6495],
          [-2.1008, -1.9124, -1.3302,  ...,  1.7009,  1.6495,  1.5639],
          [ 0.9817,  1.4440, -1.5185,  ...,  1.4440,  1.5468,  1.4954],
          ...,
          [-1.7412, -1.6727, -1.7583,  ..., -1.6727, -1.6384, -1.5357],
          [-1.7069, -1.6555, -1.7069,  ..., -1.7069, -1.6898, -1.7240],
          [-1.8268, -1.7925, -1.7069,  ..., -1.6213, -1.5870, -1.6384]],
 
         [[-1.9832,  0.5203, -1.2304,  ...,  1.8333,  1.9209,  1.8158],
          [-2.0182, -1.8256, -1.2304,  ...,  1.8683,  1.8158,  1.7283],
          [ 1.1331,  1.6057, -1.4230,  ...,  1.6057,  1.7108,  1.6583],
          ...,
          [-1.6506, -1.5805, -1.6681,  ..., -1.5805, -1.5455, -1.4405],
          [-1.6155, -1.5630, -1.6155,  ..., -1.6155, -1.5980, -1.6331],
          [-1.7381, -1.7031, -1.6155,  ..., -1.5280, -1.4930, -1.5455]],
 
         [[-1.7522,  0.7402, -1.0027,  ...,  2.0474,  2.1346,  2.0300],
          [-1.7870, -1.5953,

定义数据加载器

In [4]:
from torch.utils.data import DataLoader

# 构建训练集、验证集和测试集的 DataLoader
train_loader = DataLoader(dataset['train'], batch_size=16, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(dataset['val'], batch_size=32, shuffle=False, num_workers=0, drop_last=True)
test_loader = DataLoader(dataset['test'], batch_size=32, shuffle=False, num_workers=0, drop_last=True)


实例化U-Net模型

In [5]:
import torch
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet50
from sklearn.metrics import f1_score
import time

class UNetTrainer:
    def __init__(self, num_classes=3, lr=1e-4):
        self.model = self.build_model(num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def build_model(self, num_classes):
        model = fcn_resnet50(pretrained=False, num_classes=num_classes)
        return model

    def evaluate(self, epoch, dataloader, phase):
        if phase == "train":
            self.model.train()
        else:
            self.model.eval()

        running_loss = 0.0
        running_corrects = 0
        running_f1_score = 0.0

        for images, masks in dataloader:
            images = images.to(self.device)
            masks = masks.to(self.device)

            # 由于数据集中的mask是三维的，需要转换为二维
            masks = torch.mean(masks, dim=3, keepdim=False).long()

            self.optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                outputs = self.model(images)['out']
                preds = torch.argmax(outputs, dim=1)
                loss = self.criterion(outputs, masks)

                if phase == "train":
                    loss.backward()
                    self.optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == masks.data)
            running_f1_score += f1_score(masks.cpu().numpy().ravel(), preds.cpu().numpy().ravel(), average="macro")

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)
        epoch_f1_score = running_f1_score / len(dataloader)

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {epoch_f1_score:.4f}")

    def train(self, num_epochs, train_loader, val_loader):
        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            print("-" * 10)

            start_time = time.time()

            self.evaluate(epoch, train_loader, "train")
            self.evaluate(epoch, val_loader, "val")

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Epoch time: {elapsed_time:.4f}s")

        print("Training complete")


In [6]:
from sklearn.metrics import f1_score, jaccard_score

class UNetTrainer:
    def __init__(self, num_classes=3, lr=1e-4):
        self.model = self.build_model(num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def build_model(self, num_classes):
        model = fcn_resnet50(pretrained=False, num_classes=num_classes)
        return model

    def calculate_iou(self, pred, target):
        return jaccard_score(target.cpu().numpy().ravel(), pred.cpu().numpy().ravel(), average="macro")

    def evaluate(self, epoch, dataloader, phase):
        if phase == "train":
            self.model.train()
        else:
            self.model.eval()

        running_loss = 0.0
        running_iou = 0.0
        running_f1_score = 0.0

        for images, masks in dataloader:
            images = images.to(self.device)
            masks = masks.to(self.device)

            masks = torch.mean(masks, dim=3, keepdim=False).long()

            self.optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                outputs = self.model(images)['out']
                preds = torch.argmax(outputs, dim=1)
                loss = self.criterion(outputs, masks)

                if phase == "train":
                    loss.backward()
                    self.optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_iou += self.calculate_iou(preds, masks.data)
            running_f1_score += f1_score(masks.cpu().numpy().ravel(), preds.cpu().numpy().ravel(), average="macro")

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_iou = running_iou / len(dataloader)
        epoch_f1_score = running_f1_score / len(dataloader)

        print(f"{phase} Loss: {epoch_loss:.4f} IoU: {epoch_iou:.4f} F1: {epoch_f1_score:.4f}")

    def train(self, num_epochs, train_loader, val_loader):
        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            print("-" * 10)

            start_time = time.time()

            self.evaluate(epoch, train_loader, "train")
            self.evaluate(epoch, val_loader, "val")

            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Epoch time: {elapsed_time:.4f}s")

        print("Training complete")

    def test(self, train_loader, val_loader, test_loader):
        # Combine train and val datasets
        combined_dataset = torch.utils.data.ConcatDataset([train_loader.dataset, val_loader.dataset])
        combined_loader = torch.utils.data.DataLoader(combined_dataset, batch_size=train_loader.batch_size, shuffle=True)

        # Retrain the model on the combined dataset
        print("Retraining the model on the combined dataset")
        self.train(len(train_loader.dataset) + len(val_loader.dataset), combined_loader, test_loader)

        # Evaluate the model on the test dataset
        print("Evaluating the model on the test dataset")
        self.evaluate(0, test_loader, "test")



In [7]:
trainer = UNetTrainer()
trainer.train(20, train_loader, val_loader)
trainer.test(train_loader, val_loader, test_loader)


C:\Users\86177\miniconda3\envs\D2L\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\86177\miniconda3\envs\D2L\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/20
----------


KeyboardInterrupt: 